In [13]:
from datasets import load_dataset

ds = load_dataset("allenai/qasper")

/home/cendekiaa/anaconda3/envs/qasper_repro/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import pandas as pd
title = ds['train'][:2]['title']
abstract = ds['train'][:2]['abstract']
data = {'Title':title,'Abstract':abstract}
df = pd.DataFrame(data)
df

,Title,Abstract
0,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...
1,"PO-EMO: Conceptualization, Annotation, and Mod...",Most approaches to emotion analysis regarding ...


In [1]:
df.to_pickle('preprocessed_train.pkl')

NameError: name 'df' is not defined

In [2]:
import pandas as pd
df1 = pd.read_pickle('preprocessed_train.pkl')
df1.head(5)

,Title,Abstract
0,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...
1,"PO-EMO: Conceptualization, Annotation, and Mod...",Most approaches to emotion analysis regarding ...
2,Community Identity and User Engagement in a Mu...,A community's identity defines and shapes its ...
3,Question Answering based Clinical Text Structu...,Clinical text structuring is a critical and fu...
4,Progress and Tradeoffs in Neural Language Models,"In recent years, we have witnessed a dramatic ..."


In [3]:
df_try = df1.head(2)

In [3]:
df_try

,Title,Abstract
0,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...
1,"PO-EMO: Conceptualization, Annotation, and Mod...",Most approaches to emotion analysis regarding ...


Use GPT-4 API

In [12]:
for abstract in df_try['Abstract']:
    print(abstract)

Recognizing affective events that trigger positive or negative sentiment has a wide range of natural language processing applications but remains a challenging problem mainly because the polarity of an event is not necessarily predictable from its constituent words. In this paper, we propose to propagate affective polarity using discourse relations. Our method is simple and only requires a very small seed lexicon and a large raw corpus. Our experiments using Japanese data show that our method learns affective events effectively without manually labeled data. It also improves supervised learning results when labeled data are small.
Most approaches to emotion analysis regarding social media, literature, news, and other domains focus exclusively on basic emotion categories as defined by Ekman or Plutchik. However, art (such as literature) enables engagement in a broader range of more complex and subtle emotions that have been shown to also include mixed emotional responses. We consider em

In [13]:
from openai import OpenAI
import time

client = OpenAI(api_key='sk-I-OKo5WoYX5FnJMnZhU-JV7RkJFqZNvwBEqHORPug5T3BlbkFJ5xcABHEGB7oPodSzGTM0lEuhIejDvL_nFq-Ejgl4QA')

for abstract in df_try['Abstract']:
    time.sleep(3)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Can you generate 7 Question and Answer pairs where the answer is extracted word for word in the given context \n {}".format(abstract)
            }
        ]
    )
    print(completion.choices[0].message)

ChatCompletionMessage(content="Sure! Here are 7 question and answer pairs derived from the provided context:\n\n1. **Question:** What is a significant challenge in recognizing affective events that trigger sentiment?\n   **Answer:** The polarity of an event is not necessarily predictable from its constituent words.\n\n2. **Question:** What method do the authors propose for propagating affective polarity?\n   **Answer:** Propagate affective polarity using discourse relations.\n\n3. **Question:** What are the requirements of the proposed method?\n   **Answer:** It only requires a very small seed lexicon and a large raw corpus.\n\n4. **Question:** In what context did the experiments conducted in the paper take place?\n   **Answer:** Using Japanese data.\n\n5. **Question:** What is a notable outcome of the authors' method when there is a lack of labeled data?\n   **Answer:** It learns affective events effectively without manually labeled data.\n\n6. **Question:** How does the proposed meth

In [15]:
completion.choices[0].message.content

'Sure! Here are 7 Question and Answer pairs based on the provided context:\n\n1. **Question:** What do most approaches to emotion analysis focus on?  \n   **Answer:** Most approaches to emotion analysis regarding social media, literature, news, and other domains focus exclusively on basic emotion categories as defined by Ekman or Plutchik.\n\n2. **Question:** How does art, such as literature, contribute to emotion analysis?  \n   **Answer:** Art (such as literature) enables engagement in a broader range of more complex and subtle emotions that have been shown to also include mixed emotional responses.\n\n3. **Question:** Whose emotions do the authors consider in their analysis?  \n   **Answer:** We consider emotions as they are elicited in the reader, rather than what is expressed in the text or intended by the author.\n\n4. **Question:** What do the authors conceptualize to capture emotions in the reader?  \n   **Answer:** Thus, we conceptualize a set of aesthetic emotions that are pr

In [16]:
import re
text = completion.choices[0].message.content
qa_pattern = re.compile(r'\*\*Question:\*\* (.*?)\s+\*\*Answer:\*\* (.*?)\n', re.DOTALL)

# Extract QA pairs
qa_pairs = qa_pattern.findall(text)

# Convert to dataframe
qa_df = pd.DataFrame(qa_pairs, columns=['Question', 'Answer'])

In [19]:
import re
import pandas as pd

# Use your text source
text = completion.choices[0].message.content

# Updated regular expression to be more flexible at the end of the text
qa_pattern = re.compile(r'\*\*Question:\*\* (.*?)\s+\*\*Answer:\*\* (.*?)(?:\n|$)', re.DOTALL)

# Extract QA pairs
qa_pairs = qa_pattern.findall(text)

# Convert to dataframe
qa_df = pd.DataFrame(qa_pairs, columns=['Question', 'Answer'])

# Display the dataframe
qa_df

,Question,Answer
0,What do most approaches to emotion analysis fo...,Most approaches to emotion analysis regarding ...
1,"How does art, such as literature, contribute t...",Art (such as literature) enables engagement in...
2,Whose emotions do the authors consider in thei...,We consider emotions as they are elicited in t...
3,What do the authors conceptualize to capture e...,"Thus, we conceptualize a set of aesthetic emot..."
4,What was the outcome of the annotation with ex...,Our annotation with experts leads to an accept...
5,What was the main focus of the first emotion c...,"Finally, we conduct first emotion classificati..."
6,What was the achieved F1-micro score on the Ge...,identifying aesthetic emotions is challenging ...


In [20]:
import pandas as pd

# Sample data resembling your dataframe
data = {
    "No": [1, 1, 2, 2],
    "Abstract": ["this is abstract for paper 1", "", "this is abstract for paper 2", ""],
    "Questions": ["What is the main focus?", "What is the methodology?", "", ""],
    "Answers": ["Focus is on energy optimization", "Used genetic algorithms", "", ""]
}

# Create a pandas dataframe
df = pd.DataFrame(data)

In [4]:
import pandas as pd
import re
import time
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key='sk-I-OKo5WoYX5FnJMnZhU-JV7RkJFqZNvwBEqHORPug5T3BlbkFJ5xcABHEGB7oPodSzGTM0lEuhIejDvL_nFq-Ejgl4QA')

# List to store results
data = {
    "No": [],
    "Abstract": [],
    "Questions": [],
    "Answers": []
}

# Loop over abstracts and call the OpenAI API for each abstract
for idx, abstract in enumerate(df_try['Abstract']):
    time.sleep(5)  # To prevent hitting rate limits

    # Call OpenAI API
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"Can you generate 7 Question and Answer pairs where the answer is extracted word for word in the given context \n {abstract}"
            }
        ]
    )

    # Extract the generated text
    text = completion.choices[0].message.content

    # Updated regular expression to ensure we capture the last QA pair even if not followed by a newline
    qa_pattern = re.compile(r'\*\*Question:\*\* (.*?)\s+\*\*Answer:\*\* (.*?)(?:\n|$)', re.DOTALL)
    qa_pairs = qa_pattern.findall(text)

    # Append data to the list with structured format
    abstract_no = idx + 1
    if qa_pairs:
        for q, a in qa_pairs:
            data["No"].append(abstract_no)
            data["Abstract"].append(abstract if len(data["Abstract"]) == 0 or data["Abstract"][-1] != abstract else "")
            data["Questions"].append(q)
            data["Answers"].append(a)
    else:
        data["No"].append(abstract_no)
        data["Abstract"].append(abstract)
        data["Questions"].append("")
        data["Answers"].append("")

# Create the final DataFrame
df_final = pd.DataFrame(data)

# Display the DataFrame
(df_final)

,No,Abstract,Questions,Answers
0,1,Recognizing affective events that trigger posi...,What has a wide range of natural language proc...,Recognizing affective events that trigger posi...
1,1,,What remains a challenging problem in recogniz...,The polarity of an event is not necessarily pr...
2,1,Recognizing affective events that trigger posi...,What method do the authors propose to use for ...,Propagate affective polarity using discourse r...
3,1,,What are the requirements of the proposed meth...,A very small seed lexicon and a large raw corpus.
4,1,Recognizing affective events that trigger posi...,What did the experiments using Japanese data s...,Our method learns affective events effectively...
5,1,,How does the proposed method affect supervised...,It improves supervised learning results when l...
6,1,Recognizing affective events that trigger posi...,What is the key benefit of the proposed method...,It learns affective events effectively without...
7,2,Most approaches to emotion analysis regarding ...,What do most approaches to emotion analysis fo...,Basic emotion categories as defined by Ekman o...
8,2,,"What type of engagement does art, such as lite...",Engagement in a broader range of more complex ...
9,2,Most approaches to emotion analysis regarding ...,How are emotions considered in this analysis?,"As they are elicited in the reader, rather tha..."


In [5]:
import pandas as pd
import re
import time
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key='sk-I-OKo5WoYX5FnJMnZhU-JV7RkJFqZNvwBEqHORPug5T3BlbkFJ5xcABHEGB7oPodSzGTM0lEuhIejDvL_nFq-Ejgl4QA')

# List to store results
data = {
    "No": [],
    "Abstract": [],
    "Questions": [],
    "Answers": []
}

# Loop over abstracts and call the OpenAI API for each abstract
for idx, abstract in enumerate(df_try['Abstract']):
    time.sleep(3)  # To prevent hitting rate limits

    # Call OpenAI API
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"Can you generate 7 Question and Answer pairs where the answer is extracted word for word in the given context \n {abstract}"
            }
        ]
    )

    # Extract the generated text
    text = completion.choices[0].message.content

    # Updated regular expression to ensure we capture the last QA pair even if not followed by a newline
    qa_pattern = re.compile(r'\*\*Question:\*\* (.*?)\s+\*\*Answer:\*\* (.*?)(?:\n|$)', re.DOTALL)
    qa_pairs = qa_pattern.findall(text)

    # Append data to the list with structured format
    abstract_no = idx + 1
    if qa_pairs:
        for i, (q, a) in enumerate(qa_pairs):
            data["No"].append(abstract_no)
            # Only add the abstract for the first question-answer pair, leave empty for others
            data["Abstract"].append(abstract if i == 0 else "")
            data["Questions"].append(q)
            data["Answers"].append(a)
    else:
        data["No"].append(abstract_no)
        data["Abstract"].append(abstract)
        data["Questions"].append("")
        data["Answers"].append("")

# Create the final DataFrame
df_final = pd.DataFrame(data)

# Display the DataFrame
print(df_final)


    No                                           Abstract  \
0    1  Recognizing affective events that trigger posi...   
1    1                                                      
2    1                                                      
3    1                                                      
4    1                                                      
5    1                                                      
6    1                                                      
7    2  Most approaches to emotion analysis regarding ...   
8    2                                                      
9    2                                                      
10   2                                                      
11   2                                                      
12   2                                                      
13   2                                                      

                                            Questions  \
0   What is the main challe

In [6]:
df_final

,No,Abstract,Questions,Answers
0,1,Recognizing affective events that trigger posi...,What is the main challenge in recognizing affe...,The polarity of an event is not necessarily pr...
1,1,,What method do the authors propose to propagat...,They propose to propagate affective polarity u...
2,1,,What are the requirements of the method propos...,It only requires a very small seed lexicon and...
3,1,,What type of data was used in the experiments ...,The experiments used Japanese data.
4,1,,What does the proposed method effectively learn?,The proposed method learns affective events ef...
5,1,,How does the proposed method impact supervised...,It improves supervised learning results when l...
6,1,,What is the focus of the research presented in...,Recognizing affective events that trigger posi...
7,2,Most approaches to emotion analysis regarding ...,What do most approaches to emotion analysis fo...,basic emotion categories as defined by Ekman o...
8,2,,What broader range of emotions does art enable...,more complex and subtle emotions that have bee...
9,2,,How do the authors consider emotions?,"as they are elicited in the reader, rather tha..."


In [11]:
# Save the DataFrame to Excel with formatting
with pd.ExcelWriter('output_dynamic.xlsx', engine='xlsxwriter') as writer:
    df_final.to_excel(writer, sheet_name='Sheet1', index=False)

    # Access the workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Variables to keep track of row ranges for merging
    current_row = 1  # Start from row 2 in Excel (index 1 because row 1 has headers)
    questions_per_abstract = 7  # You have 7 Q&A per abstract

    # Dynamically calculate the range for merging
    for i in range(len(df_final) // questions_per_abstract):
        # Calculate the start and end row for merging
        start_row = current_row + 1  # Skip header (Excel row 2 for first row of data)
        end_row = start_row + questions_per_abstract - 1

        # Merge cells for "No" and "Abstract"
        worksheet.merge_range(f'A{start_row}:A{end_row}', df_final['No'][start_row-2])  # Merging the "No" column
        worksheet.merge_range(f'B{start_row}:B{end_row}', df_final['Abstract'][start_row-2])  # Merging the "Abstract" column

        # Update the current_row to move to the next abstract block
        current_row += questions_per_abstract

    # Set column widths for better readability
    worksheet.set_column('A:A', 5)   # No column
    worksheet.set_column('B:B', 30)  # Abstract column
    worksheet.set_column('C:C', 30)  # Questions column
    worksheet.set_column('D:D', 30)  # Answers column